In [1]:
import numpy as np
import csv
# 필드 목록 정의
# 비정형 필드
title_arr=[]
# 분류값을 저장할 배열 (0,1)
ctr_arr=[]
# 텍스트와 분류값이 저장된 파일 불러오기 (학습용 데이터셋)
f=open('d:/data/text/ratings_train.csv', encoding='ms949')
reader=csv.reader(f)
max_length, length, max_idx, count, count_train=0,0,0,0,0
next(reader, None) # 헤더 스킵(첫 라인을 건너뜀)
for line in reader:
    # 레코드 전체를 하는 방법
    title_arr.append(line[0]) # 텍스트 필드
    ctr_arr.append(int(line[1])) # 결과 필드
    length=len(line[0]) # 텍스트의 길이
    if max_length<length:
        max_length=length
        max_idx=count
    count+=1
    count_train+=1
    if count_train>=10000:
        break
f.close()

In [2]:
# 텍스트와 분류값이 저장된 파일 불러오기 (검증용 데이터셋)
f2=open('d:/data/text/ratings_test.csv', 'r', encoding='ms949')
reader=csv.reader(f2)
next(reader, None) # 헤더 스킵(첫 라인을 건너뜀)
count_test=0
for line in reader:
    # 레코드 전체를 하는 방법
    title_arr.append(line[0]) # 텍스트 필드
    ctr_arr.append(int(line[1])) # 결과 필드
    length=len(line[0]) # 텍스트의 길이
    if max_length<length:
        max_length=length
        max_idx=count
    count+=1
    count_test+=1
    if count_test>=10000:
        break
f2.close()

In [3]:
print('count_train :', count_train)
print('count_test :', count_test)
print('title_arr size :', len(title_arr))
print('ctr_arr size :', len(ctr_arr))
print('max_length :', max_length)

count_train : 10000
count_test : 10000
title_arr size : 20000
ctr_arr size : 20000
max_length : 144


In [4]:
from konlpy.tag import Okt

okt=Okt()
# 명사(Noun), 형용사(Adjective), 동사(Verb), 알파벳(Alpha), 부사(Adverb)
title_noun_arr=[]
for index, title in enumerate(title_arr):
    if index%1000==0:
        print('step :', index)
    title_noun_arr.append(okt.nouns(title)) # 명사만 추출
print(len(title_noun_arr))

step : 0
step : 1000
step : 2000
step : 3000
step : 4000
step : 5000
step : 6000
step : 7000
step : 8000
step : 9000
step : 10000
step : 11000
step : 12000
step : 13000
step : 14000
step : 15000
step : 16000
step : 17000
step : 18000
step : 19000
20000


In [5]:
# 벡터 모델을 불러와서 텍스트를 벡터로 변환
import gensim

MIN_NOUN_VECTOR_VALUE=-10.0
MAX_NOUN_VECTOR_VALUE=10.0
NOUN_VECTOR_SIZE=100
# 벡터 생성 함수
def generate_random_noun_vector():
    return np.random.uniform(low=MIN_NOUN_VECTOR_VALUE, high=MAX_NOUN_VECTOR_VALUE, size=(NOUN_VECTOR_SIZE,))
# word2vec 모델 불러오기
w2v_model=gensim.models.Word2Vec.load('d:/data/text/text_100.model')
title_noun_vector_arr=[] # 벡터를 저장할 리스트
for index, title_nouns in enumerate(title_noun_arr):
    if index%10000==0:
        print('step :', index)
    noun_vector_arr=[]
    for noun in title_nouns:
        try:
            noun_vector=w2v_model[noun] # 단어별로 벡터 모델 적용
        except Exception as e:
            noun_vector=generate_random_noun_vector() # 단어사전에 존재하지 않으면 기본 벡터 생성
        noun_vector_arr.append(noun_vector)
    title_noun_vector_arr.append(noun_vector_arr)

step : 0
step : 10000


In [6]:
print(title_noun_vector_arr[:1])

[[array([ 7.00788343,  2.32315639, -1.40969122, -3.84805147,  3.29545899,
       -8.90883713,  9.01968983,  1.03392477, -4.94896253, -7.80244264,
       -6.87449978, -3.40472584, -2.65472253,  5.5133203 , -7.62624073,
        3.47444128,  3.79038021, -5.81187471,  8.77789576, -6.74079621,
       -2.83244624,  2.73309272,  1.82372676,  0.59821992,  9.11019838,
       -9.69831764,  5.11724656, -3.92678061,  3.13228975,  7.15997391,
        6.67341528, -3.38749637, -0.682627  ,  9.68549935, -7.33099522,
       -2.73028487, -5.32240615, -3.40690896, -9.60176702, -2.66634534,
        3.87404446, -3.53657815, -7.68443187, -7.57807797,  8.42360889,
        6.64454624,  1.33169172,  0.62125047,  0.35773496, -8.7316495 ,
        6.2310939 , -6.65445085,  3.19792357, -1.51205631, -6.66298008,
       -1.05532875,  1.17327262,  8.17806174,  3.84374824,  3.53025782,
       -1.48479069,  0.04184469,  0.90103294,  4.83027947, -9.73185307,
       -4.87778227, -6.63978699,  3.41341984,  6.3359605 ,  0.

In [9]:
max_size, min_size, avg, count, sum_size=0,100,0,0,0
# 가장 긴 리뷰 텍스트 계산
for index, title_noun_vector in enumerate(title_noun_vector_arr):
    if len(title_noun_vector)==0:
        pass
    sum_size+=len(title_noun_vector)
    if max_size<len(title_noun_vector):
        max_size=len(title_noun_vector)
    if min_size>len(title_noun_vector):
        min_size=len(title_noun_vector)
    count+=1

TITLE_LENGTH=max_size

def generate_zero_noun_vector():
    return np.random.uniform(low=0.0, high=0.0, size=(NOUN_VECTOR_SIZE,))

# 인덱스와 요소를 하나씩 전개하여 패딩 처리
title_noun_vector_arr2=[]
for index, title_noun_vector in enumerate(title_noun_vector_arr):
    if index%1000==0:
        print('step :', index)
    while len(title_noun_vector)<TITLE_LENGTH:
        title_noun_vector.append(generate_zero_noun_vector())
    title_noun_vector=title_noun_vector[:TITLE_LENGTH]
    title_noun_vector_arr2.append(title_noun_vector)

step : 0
step : 1000
step : 2000
step : 3000
step : 4000
step : 5000
step : 6000
step : 7000
step : 8000
step : 9000
step : 10000
step : 11000
step : 12000
step : 13000
step : 14000
step : 15000
step : 16000
step : 17000
step : 18000
step : 19000


In [10]:
print(min_size, max_size)

0 37


In [11]:
# ctr to classification
# 클래스 개수
NUM_CLASSES=2
ctr_class_arr=[]
# 클래스 개수 저장
ctr_class_count=[0,0]
for index, ctr in enumerate(ctr_arr):
    if ctr==0:
        ctr_class_arr.append(0.0)
        ctr_class_count[0]+=1
    elif ctr==1:
        ctr_class_arr.append(1.0)
        ctr_class_count[1]+=1
print(ctr_class_count)
print(ctr_class_arr[:10])

[9960, 10040]
[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0]


In [12]:
# 데이터 전처리 (0.0 ~ 1.0), 각 특성의 평균을 계산
def prepare_data(arr):
    sum1=0
    for i in arr:
        sum1+=i
    arr=[float(i)/sum1 for i in arr]
    return arr
# 전처리가 필요한 필드에 대해 함수 호출

In [13]:
# 검증용 데이터셋의 개수 구하기
test_data_size=count_test
train_data_size=count_train
print(train_data_size,' ',test_data_size)
# 학습용 데이터셋과 검증용 데이터셋을 불리하는 코드
train_input=title_noun_vector_arr[0:train_data_size]
train_label=ctr_class_arr[0:train_data_size]
test_input=title_noun_vector_arr[train_data_size:]
test_label=ctr_class_arr[train_data_size:]
print('ctr_class_arr 개수 :', len(ctr_class_arr))
print('train 총 데이터 개수 :', len(train_input))
print('train 라벨 개수 :', len(train_label))
print('test 총 데이터 개수 :', len(test_input))
print('test 라벨 개수 :', len(test_label))
print('train_data_size :', train_data_size)
print('test_data_size :', test_data_size)

10000   10000
ctr_class_arr 개수 : 20000
train 총 데이터 개수 : 10000
train 라벨 개수 : 10000
test 총 데이터 개수 : 10000
test 라벨 개수 : 10000
train_data_size : 10000
test_data_size : 10000


In [16]:
# 데이터를 케라스에 맞게 변환해주는 과정
# 데이터 개수, 가로, 세로, 채널수(흑백이미지는 1, 컬러이미지는 3, 여기서는 텍스트이므로 1을 사용)
# width, height, channel
train_input=np.array(train_input)
train_input=train_input.reshape(train_input.shape[0], train_input.shape[1], NOUN_VECTOR_SIZE, 1)
test_input=np.array(test_input)
test_input=test_input.reshape(test_input.shape[0], test_input.shape[1], NOUN_VECTOR_SIZE, 1)

In [17]:
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Dense, Dropout, Flatten
from tensorflow.keras.models import Model

input_a=Input(shape=(train_input.shape[1], NOUN_VECTOR_SIZE, 1), name='input-layer') # 입력 데이터
x=Conv2D(1,(3,3),  activation='relu', padding='valid', strides=(1,1))(input_a) # 첫 번째 Conv 레이어
x=Conv2D(1,(3,3),  activation='relu')(x) # 두 번째 Conv 레이어
x=MaxPooling2D(pool_size=(2,2))(x)
x=Dropout(0.25)(x)
# fully connected network
x=Flatten()(x)
x=Dense(128, activation='relu')(x)
x=Dropout(0.5)(x)
out=Dense(1, activation='sigmoid', name='output-layer')(x) # 출력 계층 (1개의 클래스로 분류됨)
model=Model(inputs=[input_a], outputs=out) # CNN 모델이 생성됨
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) # 모델 컴파일, 손실 함수, 옵티마이저 설정

In [18]:
# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(patience=2) # 조기 종료 콜백 함수
hist=model.fit(x=train_input, y=np.array(train_label), validation_split=0.25, batch_size=256, epochs=5, verbose=1,
callbacks=[early_stopping])

Epoch 1/5
30/30 [==============================] - 7s 189ms/step - loss: 1.1687 - accuracy: 0.4983 - val_loss: 0.7074 - val_accuracy: 0.5092
Epoch 2/5
30/30 [==============================] - 4s 118ms/step - loss: 0.7242 - accuracy: 0.5045 - val_loss: 0.6938 - val_accuracy: 0.4968
Epoch 3/5
30/30 [==============================] - 4s 118ms/step - loss: 0.6967 - accuracy: 0.5154 - val_loss: 0.6937 - val_accuracy: 0.5000
Epoch 4/5
30/30 [==============================] - 3s 113ms/step - loss: 0.6948 - accuracy: 0.5161 - val_loss: 0.6930 - val_accuracy: 0.5012
Epoch 5/5
30/30 [==============================] - 3s 113ms/step - loss: 0.6948 - accuracy: 0.5068 - val_loss: 0.6936 - val_accuracy: 0.4808


In [19]:
# 평가 
scores=model.evaluate(np.array(test_input)[:100], np.array(test_label)[:100], batch_size=1, verbose=1)
print("%s : %.2f%%" % (model.metrics_names[0], scores[0]*100)) # 손실률
print("%s : %.2f%%" % (model.metrics_names[1], scores[1]*100)) # 정확도
# CNN의 최종 출력값이 필요할 경우 아래 코드 사용
m2=Model(inputs=model.input, outputs=model.get_layer('output-layer').output)
y=m2.predict(test_input)

100/100 [==============================] - 0s 2ms/step - loss: 0.6919 - accuracy: 0.5000
loss : 69.19%
accuracy : 50.00%


In [20]:
y[0:10]

array([[0.49873996],
       [0.49873996],
       [0.5150614 ],
       [0.49850777],
       [0.5025537 ],
       [0.5305687 ],
       [0.48848504],
       [0.517841  ],
       [0.5062599 ],
       [0.50747204]], dtype=float32)

In [21]:
model.save('d:/data/text')

INFO:tensorflow:Assets written to: d:/data/text\assets


In [22]:
n=911
print(model.predict(np.array(train_input)[n].reshape(1, 37, 100, 1)))
print(np.array(train_label)[n])
print(title_arr[n])

[[0.49873996]]
1.0
재밌군


In [23]:
n=5000
print(model.predict(np.array(train_input)[n].reshape(1, 37, 100, 1)))
print(np.array(train_label)[n])
print(title_arr[n+10000])

[[0.5123401]]
0.0
OOO기 영화입니다. 긴장감도 그렇게 없고 반전도 안놀랍고 시간이 아까운 영화입니다.
